# Example notebook for different normalization techniques

Notebooks for normalization (real data; no norm, zscore, quantile transform) and check decoding performance differences



Features or raw normalization?



Normalization methods available in py_neuromodulation are:

  MEAN = "mean"

  MEDIAN = "median"

  ZSCORE = "zscore"

  ZSCORE_MEDIAN = "zscore-median"

  QUANTILE = "quantile"

  POWER = "power"

  ROBUST = "robust"

  MINMAX = "minmax"`

We are going to explore the differences between no normalization, zscoring and quantile normalization.

In pre-processing, raw normalization refers to normalizing the data with respect to the past number of "normalize_samples".

In post-processing, feature normalization refers to normalizing features with respect to the past number of "normalize_samples".

### No Normalization

 ### Zscore
 The normalized array of the current sample subtracts the previous sample mean and divides it by the previous sample standard deviation.

### Quantile

Quantile normalization is a technique for making two distributions identical in statistical properties. To quantile-normalize a test distribution to a reference distribution of the same length, sort the test distribution and sort the reference distribution. The highest entry in the test distribution then takes the value of the highest entry in the reference distribution, the next highest entry in the reference distribution, and so on, until the test distribution is a perturbation of the reference distribution. [1]

#### References
[1] Quantile normalization, https://en.wikipedia.org/wiki/Quantile_normalization